In [215]:
day16input=open("input.txt")
input = day16input.readlines()

In [216]:
len(input)

4064

In [217]:
input[3163]

'\n'

In [218]:
#split input into part 1 and part 2
input_1 = input[:3163]
input_2 = input[3166:]

## Part 1: test instructions to see which can correspond to which op codes

In [219]:
input[:10]

['Before: [1, 0, 1, 3]\n',
 '9 2 1 0\n',
 'After:  [2, 0, 1, 3]\n',
 '\n',
 'Before: [0, 1, 2, 3]\n',
 '13 0 0 1\n',
 'After:  [0, 1, 2, 3]\n',
 '\n',
 'Before: [3, 3, 2, 2]\n',
 '7 0 3 1\n']

In [220]:
import re

samples = []

for index, line in enumerate(input_1):
    if index % 4 == 0:
        params = re.split("[ ,\]\[]+",line.strip())
        samples.append({"Before":([int(x) for x in params[1:5]])})
    elif index % 4 == 1:
        params = re.split("[ ]+",line.strip())
        samples[-1]["Instruction"] = [int(x) for x in params[:4]]
    elif index % 4 == 2:
        params = re.split("[ ,\]\[]+",line.strip())
        samples[-1]["After"] = [int(x) for x in params[1:5]]
    else:
        #blank line between examples
        pass
    
    

In [221]:
samples

[{'Before': [1, 0, 1, 3], 'Instruction': [9, 2, 1, 0], 'After': [2, 0, 1, 3]},
 {'Before': [0, 1, 2, 3], 'Instruction': [13, 0, 0, 1], 'After': [0, 1, 2, 3]},
 {'Before': [3, 3, 2, 2], 'Instruction': [7, 0, 3, 1], 'After': [3, 1, 2, 2]},
 {'Before': [1, 3, 2, 1], 'Instruction': [8, 3, 2, 3], 'After': [1, 3, 2, 1]},
 {'Before': [3, 3, 2, 1], 'Instruction': [14, 0, 2, 1], 'After': [3, 1, 2, 1]},
 {'Before': [2, 3, 1, 3], 'Instruction': [9, 2, 1, 3], 'After': [2, 3, 1, 2]},
 {'Before': [2, 2, 1, 1], 'Instruction': [15, 3, 3, 0], 'After': [0, 2, 1, 1]},
 {'Before': [1, 1, 2, 2], 'Instruction': [5, 1, 2, 2], 'After': [1, 1, 0, 2]},
 {'Before': [1, 1, 0, 3], 'Instruction': [10, 1, 3, 2], 'After': [1, 1, 0, 3]},
 {'Before': [2, 0, 1, 1], 'Instruction': [7, 0, 2, 0], 'After': [1, 0, 1, 1]},
 {'Before': [1, 1, 3, 1], 'Instruction': [2, 1, 0, 2], 'After': [1, 1, 1, 1]},
 {'Before': [2, 1, 2, 0], 'Instruction': [5, 1, 2, 1], 'After': [2, 0, 2, 0]},
 {'Before': [2, 2, 3, 1], 'Instruction': [0, 0, 

In [222]:
def addr(pars, reg):
    reg[pars[2]] = reg[pars[0]]+reg[pars[1]]
    return(0)

def addi(pars, reg):
    reg[pars[2]] = reg[pars[0]]+pars[1]
    return(0)

def mulr(pars, reg):
    reg[pars[2]] = reg[pars[0]]*reg[pars[1]]
    return(0)

def muli(pars, reg):
    reg[pars[2]] = reg[pars[0]]*pars[1]
    return(0)

def banr(pars, reg):
    reg[pars[2]] = reg[pars[0]] & reg[pars[1]]
    return(0)

def bani(pars, reg):
    reg[pars[2]] = reg[pars[0]] & pars[1]
    return(0)

def borr(pars, reg):
    reg[pars[2]] = reg[pars[0]] | reg[pars[1]]
    return(0)

def bori(pars, reg):
    reg[pars[2]] = reg[pars[0]] | pars[1]
    return(0)

def setr(pars, reg):
    reg[pars[2]] = reg[pars[0]]
    return(0)

def seti(pars, reg):
    reg[pars[2]] = pars[0]
    return(0)

def gtir(pars, reg):
    reg[pars[2]] = 1 if pars[0] > reg[pars[1]] else 0
    return(0)

def gtri(pars, reg):
    reg[pars[2]] = 1 if reg[pars[0]] > pars[1] else 0
    return(0)

def gtrr(pars, reg):
    reg[pars[2]] = 1 if reg[pars[0]] > reg[pars[1]] else 0
    return(0)

def eqir(pars, reg):
    reg[pars[2]] = 1 if pars[0] == reg[pars[1]] else 0
    return(0)

def eqri(pars, reg):
    reg[pars[2]] = 1 if reg[pars[0]] == pars[1] else 0
    return(0)

def eqrr(pars, reg):
    reg[pars[2]] = 1 if reg[pars[0]] == reg[pars[1]] else 0
    return(0)


In [223]:
#each code maps to its function and a list of ids it might correspond to and a list of those it can't
opcodes = {
    "addr": (addr,set([]),set([])),
    "addi": (addi,set([]),set([])),
    "mulr": (mulr,set([]),set([])),
    "muli": (muli,set([]),set([])),
    "banr": (banr,set([]),set([])),
    "bani": (bani,set([]),set([])),
    "borr": (borr,set([]),set([])),
    "bori": (bori,set([]),set([])),
    "setr": (setr,set([]),set([])),
    "seti": (seti,set([]),set([])),
    "gtir": (gtir,set([]),set([])),
    "gtri": (gtri,set([]),set([])),
    "gtrr": (gtrr,set([]),set([])),
    "eqir": (eqir,set([]),set([])),
    "eqri": (eqri,set([]),set([])),
    "eqrr": (eqrr,set([]),set([]))
}

In [224]:
len(samples)

791

In [252]:
#for each sample, try all opcodes to see which could generate the output and update the corresponding ids for that opcode
behave_like_three = 0

for ii, sample in enumerate(samples):
    start, end, pars, id = sample["Before"], sample["After"], sample["Instruction"][1:], sample["Instruction"][0]
    opcode_count = 0
    for code in opcodes:
        #print(code, code[0])
        start_copy = start[:]
        opcodes[code][0](pars,start_copy) 
        if start_copy == end:
            opcodes[code][1].add(id)
            opcode_count += 1
        else:
            opcodes[code][2].add(id)
    #check if three possibilities    
    if opcode_count >= 3:
        behave_like_three += 1
        #print(ii, sample)


In [253]:
behave_like_three

547

## Part 2: find all opcode ids and run program

In [256]:
opcodes

{'addr': (<function __main__.addr(pars, reg)>,
  {6, 12},
  {0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15}),
 'addi': (<function __main__.addi(pars, reg)>,
  {2, 6, 9, 12},
  {0, 1, 3, 4, 5, 7, 8, 10, 11, 13, 14, 15}),
 'mulr': (<function __main__.mulr(pars, reg)>,
  {2, 3, 6, 11},
  {0, 1, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15}),
 'muli': (<function __main__.muli(pars, reg)>,
  {6},
  {0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15}),
 'banr': (<function __main__.banr(pars, reg)>,
  {1, 2, 3, 4, 5, 6, 11},
  {0, 7, 8, 9, 10, 12, 13, 14, 15}),
 'bani': (<function __main__.bani(pars, reg)>,
  {3, 4, 5, 6, 11},
  {0, 1, 2, 7, 8, 9, 10, 12, 13, 14, 15}),
 'borr': (<function __main__.borr(pars, reg)>,
  {2, 6, 11},
  {0, 1, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15}),
 'bori': (<function __main__.bori(pars, reg)>,
  {2, 6},
  {0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15}),
 'setr': (<function __main__.setr(pars, reg)>,
  {2, 6, 8, 11},
  {0, 1, 3, 4, 5, 7, 9, 10, 12, 13, 14, 15}),
 'seti': (

In [257]:
#find which are possible for each oppcode
potentials = [(code, opcodes[code][1] - opcodes[code][2]) for code in opcodes]

In [258]:
potentials

[('addr', {6, 12}),
 ('addi', {2, 6, 9, 12}),
 ('mulr', {2, 3, 6, 11}),
 ('muli', {6}),
 ('banr', {1, 2, 3, 4, 5, 6, 11}),
 ('bani', {3, 4, 5, 6, 11}),
 ('borr', {2, 6, 11}),
 ('bori', {2, 6}),
 ('setr', {2, 6, 8, 11}),
 ('seti', {2, 3, 4, 6, 9, 11, 12}),
 ('gtir', {1, 3, 4, 5, 6, 8, 10}),
 ('gtri', {1, 2, 3, 4, 5, 6, 10, 14, 15}),
 ('gtrr', {1, 4, 5, 6, 7, 10, 14, 15}),
 ('eqir', {1, 2, 4, 5, 10, 11, 13, 15}),
 ('eqri', {0, 1, 3, 4, 5, 7, 10, 15}),
 ('eqrr', {1, 2, 3, 4, 5, 10, 11, 13})]

In [260]:
opcode_list=list(range(16))

In [276]:
opcode_list[6] = 'muli'
opcode_list[12] = 'addr'
opcode_list[2] = 'bori'
opcode_list[9] = 'addi'
opcode_list[11] = 'borr'
opcode_list[3] = 'mulr'
opcode_list[8] = 'setr'
opcode_list[4] = 'seti'
opcode_list[5] = 'bani'
opcode_list[0] = 'eqri'
opcode_list[1] = 'banr'
opcode_list[10] = 'gtir'
opcode_list[7] = 'gtrr'
opcode_list[13] = 'eqrr'
opcode_list[14] = 'gtri'
opcode_list[15] = 'eqir'

In [277]:
opcode_list

['eqri',
 'banr',
 'bori',
 'mulr',
 'seti',
 'bani',
 'muli',
 'gtrr',
 'setr',
 'addi',
 'gtir',
 'borr',
 'addr',
 'eqrr',
 'gtri',
 'eqir']

In [278]:
input_2

['6 0 0 3\n',
 '9 3 2 3\n',
 '6 1 0 0\n',
 '9 0 3 0\n',
 '4 1 1 2\n',
 '1 0 3 0\n',
 '6 0 3 0\n',
 '12 1 0 1\n',
 '6 0 0 2\n',
 '9 2 2 2\n',
 '4 3 3 0\n',
 '2 2 3 2\n',
 '6 2 2 2\n',
 '12 1 2 1\n',
 '8 1 0 2\n',
 '4 1 1 1\n',
 '4 2 0 0\n',
 '3 1 3 0\n',
 '6 0 1 0\n',
 '12 2 0 2\n',
 '8 2 2 1\n',
 '4 0 0 0\n',
 '4 0 0 3\n',
 '4 2 0 2\n',
 '10 3 2 3\n',
 '6 3 1 3\n',
 '6 3 1 3\n',
 '12 1 3 1\n',
 '8 1 2 0\n',
 '4 1 2 1\n',
 '4 1 3 3\n',
 '4 0 0 2\n',
 '6 3 2 2\n',
 '6 2 1 2\n',
 '12 2 0 0\n',
 '8 0 1 2\n',
 '4 1 3 0\n',
 '6 2 0 1\n',
 '9 1 0 1\n',
 '4 2 3 3\n',
 '4 1 3 3\n',
 '6 3 2 3\n',
 '12 2 3 2\n',
 '8 2 1 0\n',
 '6 1 0 3\n',
 '9 3 2 3\n',
 '4 0 2 2\n',
 '4 1 2 1\n',
 '3 1 3 2\n',
 '6 2 3 2\n',
 '12 2 0 0\n',
 '8 0 3 1\n',
 '6 3 0 2\n',
 '9 2 3 2\n',
 '4 0 0 3\n',
 '4 3 1 0\n',
 '5 0 2 3\n',
 '6 3 2 3\n',
 '12 3 1 1\n',
 '8 1 0 0\n',
 '4 3 3 1\n',
 '4 1 3 3\n',
 '4 0 2 2\n',
 '9 3 1 2\n',
 '6 2 1 2\n',
 '12 0 2 0\n',
 '8 0 0 1\n',
 '4 1 1 0\n',
 '4 0 1 3\n',
 '4 0 1 2\n',
 '6 0 2 2\

In [279]:
#initial config from puzzle text
start = [0,0,0,0]

In [280]:
for operation in input_2:
    instructions = [int(x) for x in operation.strip().split()]
    index = instructions[0]
    pars = instructions[1:]
    opcodes[opcode_list[index]][0](pars,start)
    
print(start)


[582, 6, 582, 2]


## Testing

In [250]:
sample = samples[771]

In [251]:
start, end, pars, id = sample["Before"], sample["After"], sample["Instruction"][1:], sample["Instruction"][0]
print(sample)
opcode_count = 0
for code in opcodes:
    #print(code, code[0])
    start_copy = start[:]
    opcodes[code][0](pars,start_copy) 
    if start_copy == end:
        print(code)
        opcodes[code][1].add(id)
        opcode_count += 1
    else:
        opcodes[code][2].add(id)

{'Before': [2, 3, 2, 0], 'Instruction': [13, 2, 0, 0], 'After': [1, 3, 2, 0]}
gtri
eqir
eqrr


In [241]:
sample = {'Before': [0, 1, 2, 3], 'Instruction': [13, 0, 0, 1], 'After': [0, 1, 2, 3]}

In [242]:
sample

{'Before': [0, 1, 2, 3], 'Instruction': [13, 0, 0, 1], 'After': [0, 1, 2, 3]}

In [244]:
gtri(sample['Instruction'][1:],sample['Before'])

0

In [236]:
len(samples)*4

3164

In [237]:
input[len(samples)*4+1]

'\n'

In [199]:
samples[0]

{'Before': [1, 0, 1, 3], 'Instruction': [9, 2, 1, 0], 'After': [2, 0, 1, 3]}

In [203]:
samples[-1]

{'Before': [3, 3, 2, 1], 'Instruction': [8, 3, 2, 2], 'After': [3, 3, 1, 1]}

In [206]:
input[112:124]

['Before: [1, 2, 2, 2]\n',
 '4 0 2 2\n',
 'After:  [1, 2, 0, 2]\n',
 '\n',
 'Before: [3, 1, 1, 1]\n',
 '12 2 1 1\n',
 'After:  [3, 2, 1, 1]\n',
 '\n',
 'Before: [0, 1, 1, 1]\n',
 '12 2 1 3\n',
 'After:  [0, 1, 1, 2]\n',
 '\n']